In [1]:
from pyhealth.datasets import UMLSDataset

umls_ds = UMLSDataset(
    root="https://storage.googleapis.com/pyhealth/umls/",
    dev=True
)

/home/pj20/miniconda3/envs/pyhealth/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
umls_ds.stat()


Statistics of base dataset (dev=True):
	- Dataset: UMLSDataset
	- Number of triples: 88176
	- Number of entities: 9737
	- Number of relations: 8
	- Task name: Null
	- Number of samples: 0



In [3]:
from pyhealth.tasks import link_prediction_fn

umls_ds = umls_ds.set_task(link_prediction_fn, save=False)

100%|██████████| 88176/88176 [00:18<00:00, 4829.22it/s]


In [4]:
umls_ds.stat()


Statistics of base dataset (dev=True):
	- Dataset: UMLSDataset
	- Number of triples: 88176
	- Number of entities: 9737
	- Number of relations: 8
	- Task name: link_prediction_fn
	- Number of samples: 176352



In [5]:
from pyhealth.datasets import split_by_keys, get_dataloader_kg

train_dataset, val_dataset, test_dataset = split_by_keys(umls_ds, [0.8, 0.1, 0.1])

In [6]:
train_loader = get_dataloader_kg(train_dataset, batch_size=16, shuffle=True)
val_loader = get_dataloader_kg(val_dataset, batch_size=16, shuffle=False)
test_loader = get_dataloader_kg(test_dataset, batch_size=16, shuffle=False)

In [7]:
from pyhealth.models.kg import TransE

model = TransE(
    dataset=umls_ds
)